In [13]:
import logging
from pathlib import Path

import pandas as pd
from dotenv import dotenv_values

from generator_drainage_units import run_generator_drainage_units

logging.basicConfig(level=logging.INFO)
pd.set_option("future.no_silent_downcasting", True)

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# Define case and base_dir
config = dotenv_values("..\\.env")
base_dir = Path(config["BASE_DIR"])
# case_name
# case_name = "vallei_en_veluwe"
# case_name = "geerestein"
# case_name = "hattemerbroek"
# case_name = "pangelerbeek"
case_name = "Leuvenumse_beek"
case_path = Path(base_dir, case_name)

dir_basisdata = "0_basisdata"
dir_results = "1_resultaat"
ghg_file_name = "GHG_2000-2010_L1.NC"

In [22]:
gdu = run_generator_drainage_units(
    path=case_path,
    dir_basisdata=dir_basisdata,
    dir_results=dir_results,
    ghg_file_name=ghg_file_name,
    read_results=True,
    write_results=True,
    preprocess=True,
    process=False,
    postprocess=False,
    resolution=2.0,
    depth_waterways=0.2,
    buffer_waterways=2.5,
    smooth_distance=25.0,
    iterations=12500,
    iteration_group=100,
)

INFO:root: ### Case "Leuvenumse_beek" ###
INFO:root:     - dir basisdata    = c:\Users\NLHARN\Sweco AB\51024754_Vallei_en_Veluwe_Afwateringseenheden - Documenten\General\Analyse\3_analyses\Leuvenumse_beek\0_basisdata
INFO:root:     - dir results      = c:\Users\NLHARN\Sweco AB\51024754_Vallei_en_Veluwe_Afwateringseenheden - Documenten\General\Analyse\3_analyses\Leuvenumse_beek\1_resultaat
INFO:root:   x read basisdata
INFO:root:     - get dataset HYDROOBJECTEN
INFO:root:     - get dataset OVERIGE_WATERGANGEN
INFO:root:   x read results
INFO:root:     - get dataset ALL_WATERWAYS_0
INFO:root:     - get dataset ALL_WATERWAYS_1
INFO:root:     - get dataset DRAINAGE_UNITS_0
INFO:root:     - get dataset EDGES
INFO:root:     - get dataset HYDROOBJECTEN_PROCESSED_0
INFO:root:     - get dataset NODES
INFO:root:     - get dataset OUTFLOW_NODES
INFO:root:     - get dataset OVERIGE_WATERGANGEN_PROCESSED_4
INFO:root:     - use dataset processed hydroobjecten: hydroobjecten_processed_0.gpkg
INFO:roo

In [ ]:
gdu.generate_folium_map(
    order_labels=True,
    zmin=7.5,
    zmax=25.0,
    dx=0.0,
    dy=-9.0,
)